In [6]:
import pandas as pd
import numpy as np 
import torch
from torch import nn
from sklearn.model_selection import cross_validate,KFold
from torch.optim import Adam

In [469]:
Churn =pd.read_csv('/BankChurners.csv')
Churn = Churn.drop(Churn.columns[0],axis=1)
min_valid_loss = np.inf

In [470]:
class dnn(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim,30),    
            nn.ReLU(),        # first  hidden layer
            nn.Linear(30,20),
            nn.ReLU(),        # second hidden layer
            nn.Linear(20,10),
            nn.ReLU(),        # third  hidden layer
            
            nn.Linear(10,output_dim),
            nn.Sigmoid()
            )
    def forward(self, x):
        x = x.view(x.shape[0], -1)
        return self.layers(x) 

In [471]:
dnn = dnn(26,1)

In [489]:
loss = torch.nn.BCELoss()
optimizer = torch.optim.SGD(dnn.parameters(),
                            lr=0.01,
                            weight_decay=0.002,  
                            momentum=0.9)
#optimizer = torch.optim.Adam(dnn.parameters(), lr=0.01)

In [490]:
class ChurnDataset(torch.utils.data.Dataset):
    def __init__(self,churnpd):
        churn=churnpd.copy()
 
        x=churn[Churn.columns[1:]].values
        y=churn.Attrition_Flag.values
 
        self.x_train=torch.tensor(x,dtype=torch.float32)
        self.y_train=torch.tensor(y,dtype=torch.float32)

    def __len__(self):
            
        return len(self.y_train)

    def __getitem__(self,idx):
        return self.x_train[idx],self.y_train[idx]

In [500]:
 for i in range(10): 
    kf5 = KFold(n_splits=5, shuffle=True)
    for train, test in kf.split(Churn):
        dnn.train() 
        trainset = Churn.iloc[train].copy()
        testset  = Churn.iloc[test].copy()
        trDataset = ChurnDataset(trainset)
        teDataset = ChurnDataset(testset)
        train, valid = torch.utils.data.random_split(trDataset,[trainset.shape[0]-int(trainset.shape[0]/5),int(trainset.shape[0]/5)])
        validloader = torch.utils.data.DataLoader(valid, batch_size=128, shuffle=True)
        trainloader = torch.utils.data.DataLoader(trDataset, batch_size=128, shuffle=True)
        testloader =  torch.utils.data.DataLoader(teDataset, batch_size=128, shuffle=True)
        for x,y in trainloader:
            optimizer.zero_grad() 
            output = dnn(x)  
            new_loss = loss(output.view(-1), y)  
            new_loss.backward() 
            optimizer.step()
        dnn.eval()
        valid_loss = 0.0
        for data, labels in validloader:            
            target = dnn(data)
            lossvalue = loss(target.view(-1),labels)
            valid_loss += lossvalue.item()
            res_lbl = target.view(-1)       
        
        if min_valid_loss > valid_loss:            
            min_valid_loss = valid_loss
            torch.save(dnn.state_dict(), 'best_model')
        dnn.eval() 
        torch.load('best_model')
        correct = 0
        total = 0
        with torch.no_grad():
            for x,y in testloader:
                output = dnn(x)                             
                output[output>0.5] = 1
                output[output<=0.5] = 0
                res_lbl = output.view(-1)            
                correct += (res_lbl == y).sum()
                total += y.shape[0]                
                acc = correct / total
                print(f"Accuracy for testing dataset: {acc:.3f}")

Accuracy for testing dataset: 0.891
Accuracy for testing dataset: 0.855
Accuracy for testing dataset: 0.841
Accuracy for testing dataset: 0.834
Accuracy for testing dataset: 0.855
Accuracy for testing dataset: 0.859
Accuracy for testing dataset: 0.858
Accuracy for testing dataset: 0.865
Accuracy for testing dataset: 0.871
Accuracy for testing dataset: 0.867
Accuracy for testing dataset: 0.867
Accuracy for testing dataset: 0.868
Accuracy for testing dataset: 0.868
Accuracy for testing dataset: 0.870
Accuracy for testing dataset: 0.873
Accuracy for testing dataset: 0.874
Accuracy for testing dataset: 0.969
Accuracy for testing dataset: 0.969
Accuracy for testing dataset: 0.964
Accuracy for testing dataset: 0.961
Accuracy for testing dataset: 0.961
Accuracy for testing dataset: 0.960
Accuracy for testing dataset: 0.961
Accuracy for testing dataset: 0.962
Accuracy for testing dataset: 0.963
Accuracy for testing dataset: 0.963
Accuracy for testing dataset: 0.963
Accuracy for testing dataset

Accuracy for testing dataset: 0.918
Accuracy for testing dataset: 0.919
Accuracy for testing dataset: 0.915
Accuracy for testing dataset: 0.914
Accuracy for testing dataset: 0.984
Accuracy for testing dataset: 0.977
Accuracy for testing dataset: 0.966
Accuracy for testing dataset: 0.965
Accuracy for testing dataset: 0.967
Accuracy for testing dataset: 0.967
Accuracy for testing dataset: 0.963
Accuracy for testing dataset: 0.965
Accuracy for testing dataset: 0.967
Accuracy for testing dataset: 0.962
Accuracy for testing dataset: 0.963
Accuracy for testing dataset: 0.962
Accuracy for testing dataset: 0.962
Accuracy for testing dataset: 0.960
Accuracy for testing dataset: 0.961
Accuracy for testing dataset: 0.960
Accuracy for testing dataset: 0.961
Accuracy for testing dataset: 0.953
Accuracy for testing dataset: 0.951
Accuracy for testing dataset: 0.953
Accuracy for testing dataset: 0.956
Accuracy for testing dataset: 0.956
Accuracy for testing dataset: 0.950
Accuracy for testing dataset

Accuracy for testing dataset: 0.893
Accuracy for testing dataset: 0.938
Accuracy for testing dataset: 0.941
Accuracy for testing dataset: 0.951
Accuracy for testing dataset: 0.957
Accuracy for testing dataset: 0.959
Accuracy for testing dataset: 0.960
Accuracy for testing dataset: 0.961
Accuracy for testing dataset: 0.961
Accuracy for testing dataset: 0.964
Accuracy for testing dataset: 0.966
Accuracy for testing dataset: 0.967
Accuracy for testing dataset: 0.964
Accuracy for testing dataset: 0.963
Accuracy for testing dataset: 0.964
Accuracy for testing dataset: 0.962
Accuracy for testing dataset: 0.961
Accuracy for testing dataset: 0.969
Accuracy for testing dataset: 0.938
Accuracy for testing dataset: 0.932
Accuracy for testing dataset: 0.934
Accuracy for testing dataset: 0.941
Accuracy for testing dataset: 0.938
Accuracy for testing dataset: 0.938
Accuracy for testing dataset: 0.939
Accuracy for testing dataset: 0.941
Accuracy for testing dataset: 0.939
Accuracy for testing dataset

Accuracy for testing dataset: 0.961
Accuracy for testing dataset: 0.965
Accuracy for testing dataset: 0.951
Accuracy for testing dataset: 0.951
Accuracy for testing dataset: 0.950
Accuracy for testing dataset: 0.951
Accuracy for testing dataset: 0.954
Accuracy for testing dataset: 0.955
Accuracy for testing dataset: 0.953
Accuracy for testing dataset: 0.955
Accuracy for testing dataset: 0.955
Accuracy for testing dataset: 0.956
Accuracy for testing dataset: 0.957
Accuracy for testing dataset: 0.958
Accuracy for testing dataset: 0.957
Accuracy for testing dataset: 0.958
Accuracy for testing dataset: 0.883
Accuracy for testing dataset: 0.898
Accuracy for testing dataset: 0.911
Accuracy for testing dataset: 0.914
Accuracy for testing dataset: 0.923
Accuracy for testing dataset: 0.921
Accuracy for testing dataset: 0.919
Accuracy for testing dataset: 0.915
Accuracy for testing dataset: 0.917
Accuracy for testing dataset: 0.917
Accuracy for testing dataset: 0.911
Accuracy for testing dataset